In [ ]:
## Import libraries
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
from IPython import display
import datetime
import time

In [ ]:
# webcam related class - iterators
class openwebcam:
    def __init__(self, seconds):
        self.seconds = seconds #Time to capture video
        self.vc = cv2.VideoCapture(0) #Open webcam 0:first camera, usually webcam on laptops
        self.starttime = datetime.datetime.now()
        
    def __iter__(self):
        self.elapsedtime = None
        return self
    
    def next(self):
        self.elapsedtime = (datetime.datetime.now() - self.starttime).total_seconds() # Count elapsed time
        self.plotfromwebcam()
        if self.elapsedtime > self.seconds: #Get out when enought time has passed
            raise StopIteration
        return self.elapsedtime
    
    def plotfromwebcam(self):
        if self.vc.isOpened(): 
            is_capturing, frame = self.vc.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            webcam_preview = plt.imshow(frame)   
            plt.draw()            
            display.clear_output(wait=True) #Clears output of current cell, waits until next output
            display.display(plt.gcf())
            plt.close()
            
        else:
            raise Exception('The camera has gone missing.')

    def __exit__(self):
        self.vc.release()
        
for i in openwebcam(5):
    plt.text(5, 100, 'ElapsedTime: %0.2f' %i, color='red', fontsize=14) #Display elapsed time

In [ ]:
#Plot from webcam using iterators
